In [33]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Sparkies').master('local[*]').getOrCreate()
sc = spark.sparkContext

# Crear Dataframes

In [34]:
#Crear un dataframe a partir de un RDD
#El RDD tiene una estructura de tuplas el número y el cuadrado de ese número
rdd = sc.parallelize([(1,1),(2,4),(3,9),(4,16),(5,25),(6,36),(7,49),(8,64),(9,81),(10,100)])
df = spark.createDataFrame(rdd,['numero','cuadrado'])
df.show()

+------+--------+
|numero|cuadrado|
+------+--------+
|     1|       1|
|     2|       4|
|     3|       9|
|     4|      16|
|     5|      25|
|     6|      36|
|     7|      49|
|     8|      64|
|     9|      81|
|    10|     100|
+------+--------+



In [35]:
#Crear un dataframe a partir de un RDD con schema
#El RDD tiene tuplas con el id de la persona, el nombre y la edad
rdd = sc.parallelize([(1,'Juan',20.5),(2,'Pedro',30),(3,'Maria',40.2),(4,'Luis',50.0)])

In [36]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType
#Definimos el schema de la primera forma
schema_1 = StructType([
    StructField('id',IntegerType(),True),
    StructField('nombre',StringType(),True),
    StructField('edad',DoubleType(),True)
])

In [37]:
#Definimos el schema de la segunda forma
schema_2 = "`id` INT, `nombre` STRING, `edad` DOUBLE"

In [38]:
#Creamos el dataframe 1
df1 = spark.createDataFrame(rdd,schema_1)
df1.printSchema()

root
 |-- id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- edad: double (nullable = true)



In [39]:
#Creamos el dataframe 2
df2 = spark.createDataFrame(rdd,schema_2)
df2.printSchema()

root
 |-- id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- edad: double (nullable = true)



In [40]:
#Crear un dataframe a partir de un .txt
df_txt = spark.read.text('./work/sources/dataTXT.txt')
df_txt.show(truncate=False)

+-----------------------------------------------------------------------+
|value                                                                  |
+-----------------------------------------------------------------------+
|Estamos en el curso de pyspark                                         |
|En este capítulo estamos estudiando el API SQL de Saprk                |
|En esta sección estamos creado dataframes a partir de fuentes de datos,|
|y en este ejemplo creamos un dataframe a partir de un texto plano      |
+-----------------------------------------------------------------------+



In [41]:
#Crear un dataframe a partir de un .csv
df_csv = spark.read.csv('./work/sources/dataCSV.csv',header=True)
df_csv.show(truncate=False)

+-----------+-------------+--------------------------------------------------------------------------------------+---------------------+-----------+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------

In [42]:
#Crear un dataframe a partir de un archivo con un delimitador |
df_pipe = spark.read.csv('./work/sources/dataTab.txt',sep='|',header=True)
df_pipe.show(truncate=False)

+----+----+----------+-----+
|pais|edad|fecha     |color|
+----+----+----------+-----+
|MX  |23  |2021-02-21|rojo |
|CA  |56  |2021-06-10|azul |
|US  |32  |2020-06-02|verde|
+----+----+----------+-----+



In [43]:
#Crear un dataframe a partir de un archivo .json con un schema
json_schema = StructType([
    StructField('color',StringType(),True),
    StructField('edad',IntegerType(),True),
    StructField('fecha',DateType(),True),
    StructField('pais',StringType(),True),
])

df_json = spark.read.json('./work/sources/dataJSON.json',schema=json_schema)
df_json.show(truncate=False)

+-----+----+----------+----+
|color|edad|fecha     |pais|
+-----+----+----------+----+
|rojo |NULL|2021-02-21|MX  |
|azul |NULL|2021-06-10|CA  |
|verde|NULL|2020-06-02|US  |
+-----+----+----------+----+



In [44]:
#Crear un dataframe a partir de un archivo .parquet
df_parquet = spark.read.parquet('./work/sources/dataPARQUET.parquet')
df_parquet.show(truncate=False)

+-----------+-------------+--------------------------------------------------------------------------------------+---------------------+-----------+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------

# Trabajando con Spark SQL

In [45]:
#Cargar dataframe .parquet
df_parquet = spark.read.parquet('./work/sources/dataPARQUET.parquet')
df_parquet.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [47]:
#Seleccionar columnas
df_parquet.select('title','channel_title').show(truncate=False)

+--------------------------------------------------------------------------------------+---------------------+
|title                                                                                 |channel_title        |
+--------------------------------------------------------------------------------------+---------------------+
|WE WANT TO TALK ABOUT OUR MARRIAGE                                                    |CaseyNeistat         |
|The Trump Presidency: Last Week Tonight with John Oliver (HBO)                        |LastWeekTonight      |
|Racist Superman | Rudy Mancuso, King Bach & Lele Pons                                 |Rudy Mancuso         |
|Nickelback Lyrics: Real or Fake?                                                      |Good Mythical Morning|
|I Dare You: GOING BALD!?                                                              |nigahiga             |
|2 Weeks with iPhone X                                                                 |iJustine             |
|

In [52]:
#Seleccionar columnas con col
from pyspark.sql.functions import col
df_parquet.select(col('title'), col('channel_title')).show(truncate=False)


+--------------------------------------------------------------------------------------+---------------------+
|title                                                                                 |channel_title        |
+--------------------------------------------------------------------------------------+---------------------+
|WE WANT TO TALK ABOUT OUR MARRIAGE                                                    |CaseyNeistat         |
|The Trump Presidency: Last Week Tonight with John Oliver (HBO)                        |LastWeekTonight      |
|Racist Superman | Rudy Mancuso, King Bach & Lele Pons                                 |Rudy Mancuso         |
|Nickelback Lyrics: Real or Fake?                                                      |Good Mythical Morning|
|I Dare You: GOING BALD!?                                                              |nigahiga             |
|2 Weeks with iPhone X                                                                 |iJustine             |
|

In [53]:
#Usar alias y crear una nueva columna
df_parquet.select(col('title').alias('titulo'), col('channel_title').alias('canal')).show(truncate=False)


+--------------------------------------------------------------------------------------+---------------------+
|titulo                                                                                |canal                |
+--------------------------------------------------------------------------------------+---------------------+
|WE WANT TO TALK ABOUT OUR MARRIAGE                                                    |CaseyNeistat         |
|The Trump Presidency: Last Week Tonight with John Oliver (HBO)                        |LastWeekTonight      |
|Racist Superman | Rudy Mancuso, King Bach & Lele Pons                                 |Rudy Mancuso         |
|Nickelback Lyrics: Real or Fake?                                                      |Good Mythical Morning|
|I Dare You: GOING BALD!?                                                              |nigahiga             |
|2 Weeks with iPhone X                                                                 |iJustine             |
|

In [54]:
#Seleccionar columnas con selectExpr
df_parquet.selectExpr('title as titulo','channel_title as canal').show(truncate=False)

+--------------------------------------------------------------------------------------+---------------------+
|titulo                                                                                |canal                |
+--------------------------------------------------------------------------------------+---------------------+
|WE WANT TO TALK ABOUT OUR MARRIAGE                                                    |CaseyNeistat         |
|The Trump Presidency: Last Week Tonight with John Oliver (HBO)                        |LastWeekTonight      |
|Racist Superman | Rudy Mancuso, King Bach & Lele Pons                                 |Rudy Mancuso         |
|Nickelback Lyrics: Real or Fake?                                                      |Good Mythical Morning|
|I Dare You: GOING BALD!?                                                              |nigahiga             |
|2 Weeks with iPhone X                                                                 |iJustine             |
|

In [55]:
#Count video_id y renombrar la columna como videos
df_parquet.selectExpr('count(video_id) as videos').show(truncate=False)

+------+
|videos|
+------+
|48137 |
+------+



In [58]:
#Transformaciones filter y where
df_parquet.filter(col('video_id') == 'n1WpP7iowLc').show(truncate=False)

+-----------+-------------+------------------------------------------+-------------+-----------+------------------------+---------------------------------------------------------------+--------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|video_id   |trending_date|title                                     |chann

In [59]:
df_parquet.where(col('video_id') == 'n1WpP7iowLc').show(truncate=False)

+-----------+-------------+------------------------------------------+-------------+-----------+------------------------+---------------------------------------------------------------+--------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|video_id   |trending_date|title                                     |chann

In [60]:
#Transformaciones distinct
df_parquet.select('channel_title').distinct().show(truncate=False)


+----------------------------------------------------------------------------------------+
|channel_title                                                                           |
+----------------------------------------------------------------------------------------+
|90s Commercials                                                                         |
|David Jardine                                                                           |
|The New York Times                                                                      |
|National Safety Council                                                                 |
|SPORTSNET                                                                               |
|Paul Sellers                                                                            |
|NBC                                                                                     |
|AnselElgortVEVO                                                                         |

In [61]:
#Transformaciones dropDuplicates
df_parquet.select('channel_title').dropDuplicates().show(truncate=False)

+----------------------------------------------------------------------------------------+
|channel_title                                                                           |
+----------------------------------------------------------------------------------------+
|90s Commercials                                                                         |
|David Jardine                                                                           |
|The New York Times                                                                      |
|National Safety Council                                                                 |
|SPORTSNET                                                                               |
|Paul Sellers                                                                            |
|NBC                                                                                     |
|AnselElgortVEVO                                                                         |

In [62]:
#Transformaciones orderBy
df_parquet.select('channel_title').dropDuplicates().orderBy('channel_title').show(truncate=False)

+----------------------------------------------------------------------------------------------------------------+
|channel_title                                                                                                   |
+----------------------------------------------------------------------------------------------------------------+
|NULL                                                                                                            |
| & trying new things and adventures in new lands!                                                               |
| 2018. (Joel Bissell | MLive.com)"                                                                              |
| Brie Larson                                                                                                    |
| Lorenzo’s                                                                                                      |
| Michigan; 19-year-old Syrian refugee and advocate for girls' education Muzoon 

In [63]:
#Transformaciones sort
df_parquet.select('channel_title').dropDuplicates().sort('channel_title').show(truncate=False)

+----------------------------------------------------------------------------------------------------------------+
|channel_title                                                                                                   |
+----------------------------------------------------------------------------------------------------------------+
|NULL                                                                                                            |
| & trying new things and adventures in new lands!                                                               |
| 2018. (Joel Bissell | MLive.com)"                                                                              |
| Brie Larson                                                                                                    |
| Lorenzo’s                                                                                                      |
| Michigan; 19-year-old Syrian refugee and advocate for girls' education Muzoon 

In [64]:
#Transformaciones withColumn
df_parquet.withColumn('likes_por_vistas',col('likes')/col('views')).show(truncate=False)

+-----------+-------------+--------------------------------------------------------------------------------------+---------------------+-----------+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------

In [65]:
#Transformaciones withColumnRenamed
df_parquet.withColumnRenamed('likes','likes_totales').show(truncate=False)

+-----------+-------------+--------------------------------------------------------------------------------------+---------------------+-----------+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+-------------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------

In [66]:
#Drop
df_parquet.drop('likes').show(truncate=False)

+-----------+-------------+--------------------------------------------------------------------------------------+---------------------+-----------+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------

In [67]:
#Dropna
df_parquet.dropna().show(truncate=False)

+-----------+-------------+--------------------------------------------------------------------------------------+---------------------+-----------+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------

In [68]:
#Fillna
df_parquet.fillna(0).show(truncate=False)

+-----------+-------------+--------------------------------------------------------------------------------------+---------------------+-----------+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------

In [72]:
#Sample
#Muestra el 10% de los datos
df_parquet.sample(False,0.1).show(truncate=False)

+------------------------+-------------+--------------------------------------------------------------------------------------------------+---------------------------------------+-----------+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+--------------------------------------------------------------------------------------------

In [73]:
#Muestra el 10% de los datos con una semilla
df_parquet.sample(False,0.1,123).show(truncate=False)

+-----------+-------------+--------------------------------------------------------------------------------------------------+----------------------------------+-----------+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+--------------------------------------------------------------------------------------------

In [75]:
#Muestra el 10% de los datos con una semilla y reemplazando
df_parquet.sample(True,0.1,123).show(truncate=False)

+-----------------------------------------------------------------------------------------------------------+---------------+------------------------------------------------------------------------------------+------------------------------------+----------------+------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+----------

In [76]:
#randomSplit
#Divide el dataframe en dos dataframes con una semilla
df1, df2 = df_parquet.randomSplit([0.5,0.5],123)